In [ ]:
%load_ext autoreload

%autoreload 2

import pickle
from datetime import datetime

import pandas as pd
from rich import print as rprint

from scheduler.config import YAML_CONFIG
from scheduler.exam_proctor import Exam, Proctor
from scheduler.path import OUTPUTS_DIR
from scheduler.planner import Planner
from scheduler.prep_data import Parser, Prepper
from scheduler.simulator import Simulator
from scheduler.utils import init_logger

pd.set_option("display.max_columns", None)

In [ ]:
# Initialize logger
init_logger("logs.log")

In [ ]:
parser = Parser(YAML_CONFIG)
prepper = Prepper(*parser.parse(), YAML_CONFIG)
prepper.prepare(auto_add=False)

In [ ]:
number_of_simulations = 5000

planner = Planner(prepper.exams, prepper.proctors)
simulator = Simulator(planner, number_of_simulations)
simulator.simulate()
simulator.measure_fairness_all()
ordered_by_fairness = simulator.order_by_fairness()

In [ ]:
col_0 = "SIMULATION NUMBER"
col_1 = "TOTAL MAX-MIN"
col_2 = "TOTAL STDEV 1ST"
col_3 = "TOTAL STDEV ALL"
print(f"{col_0:22}{col_1:16}{col_2:16}{col_3:16}")
for sim in ordered_by_fairness[:5]:
    val_0 = sim
    val_1 = simulator.fairness_results[sim][1]
    val_2 = simulator.fairness_results[sim][2]
    val_3 = simulator.fairness_results[sim][3]
    print(f"{val_0:<22}{val_1:<16}{val_2:<16.2f}{val_3:<16.2f}")

In [ ]:
name1, name2 = "", ""

sim_numbers = []
for i in simulator.results:
    proctors = simulator.results[i][2]
    data = []
    for proctor in proctors:
        if proctor.name == name1:
            data.append(set(exam.code for exam in proctor.duties))
        if proctor.name == name2:
            data.append(set(exam.code for exam in proctor.duties))
    if data[0] == data[1]:
        sim_numbers.append(i)

sim_numbers = sorted(sim_numbers, key=lambda x: ordered_by_fairness.index(x))

col_0 = "SIMULATION NUMBER"
col_1 = "TOTAL MAX-MIN"
col_2 = "TOTAL STDEV 1ST"
col_3 = "TOTAL STDEV ALL"
print(f"{col_0:22}{col_1:16}{col_2:16}{col_3:16}")
for sim in sim_numbers[:5]:
    val_0 = sim
    val_1 = simulator.fairness_results[sim][1]
    val_2 = simulator.fairness_results[sim][2]
    val_3 = simulator.fairness_results[sim][3]
    print(f"{val_0:<22}{val_1:<16}{val_2:<16.2f}{val_3:<16.2f}")

In [ ]:
investigate = 3835

_, exams, proctors, blocks = simulator.results[investigate]

In [ ]:
for proctor in proctors:
    if proctor.name == name1:
        print(proctor.name)
        for exam in proctor.duties:
            print(exam.title, exam.classroom)
    if proctor.name == name2:
        print(proctor.name)
        for exam in proctor.duties:
            print(exam.title, exam.classroom)
    

In [ ]:
grand_total = {proctor: len(proctor.duties) + proctor.total_proctored_before for proctor in proctors}
sorted_grand_total = sorted(grand_total.items(), key=lambda item: item[1])
col_0 = "PROCTOR"
col_1 = "TOTAL DUTIES"
col_2 = "TOTAL BEFORE"
col_3 = "GRAND TOTAL"
print(f"{col_0:22}{col_1:15}{col_2:15}{col_3:15}")
for proctor, duties in sorted_grand_total:
    print(f"{proctor.name:22}{len(proctor.duties)}{proctor.total_proctored_before:15}{duties:15}")

## PICKLES

In [ ]:
def dump_to_pickle(obj: list[Exam] | list[Proctor], name: str) -> None:
    with open(OUTPUTS_DIR / "pickles" / f"{name}.pkl", "wb") as file:
        pickle.dump(obj, file)

dump = False

if dump:
    # dump to pickle
    description = f"{investigate}_best_of_the_bests"

    dump_to_pickle(exams, f"exams_{description}")
    dump_to_pickle(proctors, f"proctors_{description}")
    # dump_to_pickle(blocks, f"blocks_{description}")

In [ ]:
def read_from_pickle(name: str) -> list[Exam] | list[Proctor]:
    with open(OUTPUTS_DIR / "pickles" / f"{name}.pkl", "rb") as file:
        return pickle.load(file)

read = False

if read:
    exams: list[Exam] = read_from_pickle(f"exams_{description}")
    proctors: list[Proctor] = read_from_pickle(f"proctors_{description}")

In [ ]:
for proctor in proctors:
    if proctor.name == name1:
        print(proctor.name)
        for exam in proctor.duties:
            print(exam.title, exam.classroom)
    if proctor.name == name2:
        print(proctor.name)
        for exam in proctor.duties:
            print(exam.title, exam.classroom)

In [ ]:
grand_total = {proctor: len(proctor.duties) + proctor.total_proctored_before for proctor in proctors}
sorted_grand_total = sorted(grand_total.items(), key=lambda item: item[1])
col_0 = "PROCTOR"
col_1 = "TOTAL DUTIES"
col_2 = "TOTAL BEFORE"
col_3 = "GRAND TOTAL"
print(f"{col_0:22}{col_1:15}{col_2:15}{col_3:15}")
for proctor, duties in sorted_grand_total:
    print(f"{proctor.name:22}{len(proctor.duties)}{proctor.total_proctored_before:15}{duties:15}")

## GENERATE EXCELS

## Proctors

### Fine Tune

In [ ]:
proc = [proctor for proctor in proctors if proctor.name == name1][0]
proc.unavailable = []
proc.proctor_class = 1

### Generate

In [ ]:
data_of_interest = ["Name", "Email", "Year", "Total Before", "Total Duties", "Grand Total", "Duties"]
all_blocks_sorted = sorted(blocks)
col_names = data_of_interest + all_blocks_sorted

rows = []
for proctor in proctors:
    name = proctor.name
    email = proctor.email
    year = "Master 1" if proctor.proctor_class == 1 else "Master 2" if proctor.proctor_class == 2 else "PhD"
    total_before = proctor.total_proctored_before
    total_duties = len(proctor.duties)
    grand_total = total_before + total_duties
    duties = "\n".join([f"{exam.title} {exam.classroom}" for exam in proctor.duties])
    first_part = [name, email, year, total_before, total_duties, grand_total, duties]

    second_part = []
    exam_block_and_title = {exam.block:exam.title for exam in proctor.duties}
    
    for block in all_blocks_sorted:
        if block in proctor.unavailable:
            second_part.append("1")
        elif block in proctor.not_preferred:
            second_part.append("2")
        elif block in exam_block_and_title:
            second_part.append(exam_block_and_title[block])
        else:
            second_part.append("")
    
    rows.append(first_part + second_part)

df = pd.DataFrame(rows, columns=col_names).sort_values(by=["Year", "Grand Total", "Name"], ascending=[True, False, True]).reset_index(drop=True)
display(df)
    

In [ ]:
df.to_excel(OUTPUTS_DIR / f"proctors_final.xlsx", index=False, header=True)

## Exams

### Generate

In [ ]:
col_names = ["Exam Title", "Exam Date", "Reserved Slots", "Classroom", "Instructors", "Number of Proctors Needed", "Requires PhD Proctor", "Requires Specific Proctors", "Proctors Assigned"]

rows = []
for exam in exams:
    if len(exam.requires_specific_proctor) > 0:
        specific_proctor = ", ".join([proctor.name for proctor in exam.requires_specific_proctor])
    else:
        specific_proctor = ""
    assigned_proctors = ", ".join([proctor.name for proctor in exam.proctors])
    requires_phd_proctor = "Yes" if exam.requires_phd_proctor else ""
    rows.append((exam.title, exam.date, exam.time, exam.classroom, exam.instructor, exam.number_of_proctors_needed, requires_phd_proctor, specific_proctor, assigned_proctors))

df = pd.DataFrame(rows, columns=col_names).sort_values(by=["Exam Date", "Reserved Slots", "Instructors"], ascending=[True, True, True]).reset_index(drop=True)
display(df)

In [ ]:
df.to_excel(OUTPUTS_DIR / f"exams_final.xlsx", index=False, header=True)

## Emails

In [ ]:
def format_date(date_str: str) -> str:
    # Parse the input date string into a datetime object
    date = datetime.strptime(date_str, '%Y-%m-%d')

    # Format the date as 'Month Day, Year, Weekday'
    formatted_date = date.strftime('%B %d, %Y, %A')

    return formatted_date

chair = "Emin Karagözoğlu"

nl = '\n'

def generate_email_to_and_body(exams: list[Exam]) -> str:
    proctors = [proctor for exam in exams for proctor in exam.proctors]
    row_1 = "Course Title:"
    row_2 = "Date:"
    row_3 = "Time:"
    row_4 = "Classroom(s):"
    row_5 = "Instructor(s):"
    
    to_field = ", ".join([proctor.email for proctor in proctors])

    if len(proctors) == 1:
        salutation = f"Dear {proctors[0].name},"
    else:
        salutation = "Dear All,"
    
    classroom_details = [f"{exam.classroom}: {', '.join([proctor.name for proctor in exam.proctors])}" for exam in exams]
        
    body = f"""{salutation}

We need your services as proctor for the following exam.

Thank you,

{chair}
Chair

{row_1} {exams[0].title}
{row_2} {format_date(exams[0].date)}
{row_3} {exams[0].time}
{row_4} {', '.join([exam.classroom for exam in exams])}
{row_5} {exams[0].instructor}

{nl.join(classroom_details)}
"""
    return to_field, body

### Generate

In [ ]:
data: dict[str, dict[tuple[str, str, str], list[Exam]]] = {}

for exam in exams:
    exam_title = exam.title
    exam_date = exam.date
    exam_time = exam.time
    exam_instrucor = exam.instructor
    key1 = exam.title
    key2 = exam.date, exam.time, exam.instructor
    if key1 not in data:
        data[key1] = {}
    if key2 not in data[key1]:
        data[key1][key2] = []
    data[key1][key2].append(exam)


In [ ]:
for key in data:
    for value in data[key].values():
        to_field, body = generate_email_to_and_body(value)
        print(to_field)
        print("")
        print(body)
    break

In [ ]:
col_names = ["Exam Title", "Exam Date", "Reserved Slots", "Instructors", "Email Subject", "Email To", "Email Body"]

rows = []

for exam_title in data:
    for exam_date, exam_time, exam_instructor in data[exam_title]:
        exams = data[exam_title][exam_date, exam_time, exam_instructor]
        subject = f"Proctoring: {exam_title} on {format_date(exam_date)} at {exam_time}"
        to_field, body = generate_email_to_and_body(exams)
        rows.append((exam_title, exam_date, exam_time, exam_instructor, subject, to_field, body))

df = pd.DataFrame(rows, columns=col_names).sort_values(by=["Exam Date", "Reserved Slots", "Exam Title", "Instructors"], ascending=[True, True, True, True]).reset_index(drop=True)
display(df)

In [ ]:
df.to_excel(OUTPUTS_DIR / f"emails_final.xlsx", index=False, header=True)